It is only a simple project to apply what I learn in Python and its libraries.
Feel free to comment, criticise or suggest anything to the code.

In [7]:
from bs4 import BeautifulSoup
import re
import codecs
from datetime import datetime

Step 1:
- Load the bookmarks/starlist data from html file.

In [8]:
fname = "C:\\DiskC_Data Stars\\mozilla_bookmarks\\latestBookmark.html"
file = codecs.open(fname,'r','utf-8').read()
soup = BeautifulSoup(file,'lxml')
del soup['icon']
bookmarks_raw=soup.findAll("a")

Step 2:
- Convert bs4.element.ResutlSet object into list.
- Remove unnecesary attribute such as ```icon="..."``` attribute in ```<a></a>``` tag as it will consume a lot of spaces.  

In [9]:
bookmarks = list(bookmarks_raw)
for item in bookmarks:
    del item['icon']

Step 3:
- Extract all attributes data from each ```<a></a>``` tag for bookmark link and store in list
- Then, load each record in a list.

In [10]:
def dict_to_record(item):
    record={}
    record['Title']=str(item.contents[0])
    record.update(item.attrs)
    for key in record:
        if key == 'add_date' or key=='last_modified':
            ts=int(record[key])
            record[key]=datetime.utcfromtimestamp(ts).\
                          strftime('%Y-%m-%d %H:%M:%S')
    return record

book_list = []
for item in bookmarks:
    book_dict = dict_to_record(item)
    book_list.append(book_dict)


- Check for the length of the bookmark list.

In [11]:
len(book_list)

535

In [12]:
import pandas as pd
from IPython.display import display
import numpy as np

Step 4: 
- Load the bookmarks list into Pandas Dataframe.

In [13]:
starlist_df = pd.DataFrame(data=book_list)

- All column names can be viewed by run the code snippet below. (There are some other ways to check it, feel free to try the alternative method.)

In [14]:
starlist_df.columns.tolist()

['Title',
 'href',
 'add_date',
 'last_modified',
 'icon_uri',
 'tags',
 'last_charset']

Step 5:
- [ **Only for Mozilla Browser** ] Drop any unnecessary column from the dataframe created. For this case, it's the ```last_charset``` column.

In [16]:
if 'last_charset' in starlist_df.columns.tolist():
    starlist_df=starlist_df.drop(columns='last_charset')

In [17]:
starlist_df.head(3)

,Title,href,add_date,last_modified,icon_uri,tags
0,What is the biggest change you made in your li...,https://www.reddit.com/r/getdisciplined/commen...,2021-07-16 15:36:38,2022-01-04 14:21:51,https://www.redditstatic.com/desktop2x/img/fav...,therapy_advices_tips
1,VIA Character Strengths Survey Results | VIA I...,https://www.viacharacter.org/survey/surveys/fi...,2021-07-21 03:49:25,2022-01-04 14:21:51,https://via-static.global.ssl.fastly.net/favic...,therapy_advices_tips
2,Big Five Personality Test Results,https://openpsychometrics.org/tests/IPIP-BFFM/...,2021-07-21 03:50:40,2022-01-04 14:21:51,NaN,therapy_advices_tips


- Just want to check the link amount for each tags

In [23]:
#starlist_df.groupby('tags').count()[['Title']]

Bonus:
- Check the validity of each url link. Some of them might not be available anymore.

In [22]:
from urllib.request import Request,urlopen
from urllib.error import URLError, HTTPError

def test_the_url(link):
   req = Request(link)
   try:
      response = urlopen(req)
   except HTTPError as e:
      print('The server couldn\'t fulfill the request.')
      print('Error code: ', e.code)
      print()
   except URLError as e:
      print('We failed to reach a server.')
      print('Reason: ', e.reason)
      print()
   else:
      print('Website is working fine\n')
